In [ ]:
# This makes a cube of FITS files, so as to inspect images en masse with DS9

# Saved as .py file, 2017 Nov 19

# created 2017 Nov 19 by E.S.

In [1]:
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import numpy as np
import pandas as pd
import datetime
from astropy.io import fits
import asciitable
from pathlib import Path
import pickle
import astropy
from astropy import units as u
from astropy.coordinates import SkyCoord
import math
import itertools

In [2]:
# set file paths and datasets

dirTreeStem = ('/home/../../media/unasemaje/Seagate Expansion Drive/lbti_data_reduction/'
               'lmircam_astrometry/ut_2017_11_06/asterism/processedData/')
retrievalPiece = ('step03_derotate/')
depositPiece = ('step04_ditherMedians/')
fileNameStem = ('lm_171106_')

In [3]:
picklefile = 'identified_stars_171106.p'
fo1=open(picklefile,'rb')
dat=pickle.load(fo1)
fo1.close()

In [4]:
# read in text file

df = pd.read_csv('star_ra_dec_list.csv')

In [5]:
def order_Y(star1input, star2input, star1nameInput, star2nameInput):
    
    # turn (x,y) dataframe into numpy array
    #xy_coord = np.copy(datInput[2])
    
    # determine which object is higher in y
    if (star1input[1] > star2input[1]):
        c_high = star1input
        c_low = star2input
        name_high = star1nameInput
        name_low = star2nameInput
    else:
        c_high = star2input
        c_low = star1input
        name_high = star2nameInput
        name_low = star1nameInput
        
    # find angle (in deg)
    del_x = -np.subtract(c_high[0],c_low[0]) # minus sign because we want del_x E of N
    del_y = np.subtract(c_high[1],c_low[1])
    pos_angle_XY = np.arctan(np.divide(del_x,del_y))*(180./np.pi)
        
    return c_low, c_high, pos_angle_XY, name_low, name_high

In [6]:
# find all possible baseline permutations from the stars identified in each dither median

baselineNumber = 0 # for counting number of baselines
angleRadecMinusXYArray = [] # for collecting angle differences

# loop over dither positions
for ditherPos in range(0,len(dat.keys())):
    
    keyName = "dither_pos_"+"%02i"%ditherPos # key for this dither
    parent = dat[keyName][1] # N star names

    # list of all combinations of names (no degeneracies)
    allCombs = list(itertools.combinations(parent, 2))
    #print(allCombs)
    
    # loop over all baselines (i.e., take N, pick 2)
    for baseline in range(0,len(allCombs)):
        
        
        ## find position angle in (x,y) space
        
        star1name = allCombs[baseline][0]
        star2name = allCombs[baseline][1]
        
        names = np.array(dat[keyName][1][:])
        star1elem = int(np.where(names==star1name)[0])
        star2elem = int(np.where(names==star2name)[0])
        
        star1coords_xy = [dat[keyName][2]['[x]'][star1elem],dat[keyName][2]['[y]'][star1elem]]
        star2coords_xy = [dat[keyName][2]['[x]'][star2elem],dat[keyName][2]['[y]'][star2elem]]
        
        # sort according to y-position
        coords_low, coords_high, pos_angle_xy, name_low, name_high = order_Y(
            star1coords_xy, star2coords_xy, star1name, star2name)
        
        
        ## find position angle in (RA,DEC) space
        # note that stars are in same order (which is important if they have nearly equal DEC or y)
        
        # extract RA, DEC
        radecStarElem_low = np.where(df[' shorthand']==' '+name_low)[0] # find element number
        radecStarElem_high = np.where(df[' shorthand']==' '+name_high)[0]
        raString_low = df[' RA'][radecStarElem_low].values
        decString_low = df[' DEC'][radecStarElem_low].values
        raString_high = df[' RA'][radecStarElem_high].values
        decString_high = df[' DEC'][radecStarElem_high].values
        
        c_low = SkyCoord(raString_low+decString_low, unit=(u.hourangle, u.deg))
        c_high = SkyCoord(raString_high+decString_high, unit=(u.hourangle, u.deg))
        pos_angle_radec = c_low.position_angle(c_high).degree[0]
        
        baselineNumber += 1 # chalk up this baseline to the total
        
        #print(pos_angle_xy)
        #print(pos_angle_radec)
        
        angleDiff = np.mod(
            np.subtract(pos_angle_radec,pos_angle_xy),
            360.) # mod is in case one angle is <0 and the other >180
        
        np.append(angleRadecMinusXYArray,angleDiff) # append del_angle to array
        
        print(angleDiff) 
        
        print('-----------')
        
    print('===========')
print(baselineNumber)

0.496824888934
-----------
0.555777982607
-----------
0.585935425754
-----------
3


In [ ]:
##################################################################

In [5]:
# extract RA, DEC for C1/2 and D1

chooseStar1 = np.where(df[' shorthand']==' '+dat['dither_pos_00'][1][0])[0]
chooseStar2 = np.where(df[' shorthand']==' '+dat['dither_pos_00'][1][2])[0]
raString1 = df[' RA'][chooseStar1].values
decString1 = df[' DEC'][chooseStar1].values
raString2 = df[' RA'][chooseStar2].values
decString2 = df[' DEC'][chooseStar2].values

In [80]:
c_C12 = SkyCoord(raString1+decString1, unit=(u.hourangle, u.deg))
c_zeta = SkyCoord(raString2+decString2, unit=(u.hourangle, u.deg))
pos_angle_RADEC = c_low.position_angle(c_high).degree

In [81]:
# fcn for a single baseline

def order_DEC():
    # determine which object is higher in DEC
    if (c_C12.dec > c_zeta.dec):
        c_high = c_C12
        c_low = c_zeta
    else:
        c_high = c_zeta
        c_low = c_C12
        
    pos_angle_RADEC = c_low.position_angle(c_high).degree
        
    return c_low, c_high, pos_angle_RADEC


In [82]:
order_DEC()

(<SkyCoord (ICRS): (ra, dec) in deg
     [( 83.81858417, -5.38968978)]>, <SkyCoord (ICRS): (ra, dec) in deg
     [( 83.81977875, -5.38788269)]>, array([ 33.35034613]))

In [8]:
# fcn for a single baseline
'''
def order_Y():
    # determine which object is higher in y
    if (xy_coord[0,1] > xy_coord[2,1]):
        c_high = xy_coord[0,:]
        c_low = xy_coord[2,:]
    else:
        c_high = xy_coord[2,:]
        c_low = xy_coord[0,:]
        
    # find angle (in deg)
    del_x = -np.subtract(c_high[0],c_low[0]) # minus sign because we want del_x E of N
    del_y = np.subtract(c_high[1],c_low[1])
    pos_angle_XY = np.arctan(np.divide(del_x,del_y))*(180./np.pi)
        
    return c_low, c_high, pos_angle_XY
'''

In [7]:
# turn (x,y) dataframe into numpy array
xy_coord = np.copy(dat['dither_pos_00'][2])

In [9]:
angle_RADEC = order_DEC()[2][0]
angle_XY = order_Y()[2]

In [10]:
print(angle_RADEC)
print(angle_XY)
print(np.subtract(angle_RADEC,angle_XY))

33.3503461335
32.7945681509
0.555777982607


Maire+ 2015 (40 baselines): rotate 0.43 deg W of N
2016B (101 baselines): rotate 1.1 deg W of N
2017B (1 baseline): (rotate 0.6 deg W of N)

In [14]:
xy_coord[:,1]

array([ 359.35760981,  962.2084715 ,  969.1675042 ])

In [17]:
parent = dat['dither_pos_00'][1]

In [25]:
dat['dither_pos_00']

[[0, 1, 2], ['C1/2', 'D1', 'zeta'],            [x]         [y]
 0  1349.179608  359.357610
 1   244.937658  962.208471
 2   956.265767  969.167504]

In [7]:
# turn (x,y) dataframe into numpy array
xy_coord = np.copy(dat['dither_pos_00'][2])

In [6]:
def order_DEC(c_1,c_2):
    # determine which object is higher in DEC
    
    if (c_1.dec > c_2.dec):
        c_high = c_1
        c_low = c_2
    else:
        c_high = c_2
        c_low = c_1
        
    pos_angle_RADEC = c_low.position_angle(c_high).degree
        
    return c_low, c_high, pos_angle_RADEC

In [8]:
c1 = SkyCoord(raString1+decString1, unit=(u.hourangle, u.deg))
c2 = SkyCoord(raString2+decString2, unit=(u.hourangle, u.deg))

In [9]:
order_DEC(c1,c2)

(<SkyCoord (ICRS): (ra, dec) in deg
     [( 83.81858417, -5.38968978)]>, <SkyCoord (ICRS): (ra, dec) in deg
     [( 83.81977875, -5.38788269)]>, array([ 33.35034613]))

In [69]:
'''
def order_Y(datInput):
    
    # turn (x,y) dataframe into numpy array
    xy_coord = np.copy(datInput[2])
    
    # determine which object is higher in y
    if (xy_coord[0,1] > xy_coord[2,1]):
        c_high = xy_coord[0,:]
        c_low = xy_coord[2,:]
    else:
        c_high = xy_coord[2,:]
        c_low = xy_coord[0,:]
        
    # find angle (in deg)
    del_x = -np.subtract(c_high[0],c_low[0]) # minus sign because we want del_x E of N
    del_y = np.subtract(c_high[1],c_low[1])
    pos_angle_XY = np.arctan(np.divide(del_x,del_y))*(180./np.pi)
        
    return c_low, c_high, pos_angle_XY
'''

In [70]:
order_Y(dat['dither_pos_00'])

(array([ 1349.17960809,   359.35760981]),
 array([ 956.26576679,  969.1675042 ]),
 32.794568150893994)

In [72]:
np.copy(dat['dither_pos_00'][2])

array([[ 1349.17960809,   359.35760981],
       [  244.93765807,   962.2084715 ],
       [  956.26576679,   969.1675042 ]])

In [22]:
dat['dither_pos_00'][1][np.where(dat['dither_pos_00'][1][:]=='D1')]

TypeError: list indices must be integers or slices, not tuple

In [62]:
dat['dither_pos_00'][2]['[x]'][0]

1349.179608092631

In [40]:
test

array(['C1/2', 'D1', 'zeta'], 
      dtype='<U4')

In [47]:
print(np.where(test=='zeta')[0]

2


In [66]:
dat['dither_pos_00']

[[0, 1, 2], ['C1/2', 'D1', 'zeta'],            [x]         [y]
 0  1349.179608  359.357610
 1   244.937658  962.208471
 2   956.265767  969.167504]

In [7]:

# turn (x,y) dataframe into numpy array
xy_coord = np.copy(dat['dither_pos_00'][2])